In [101]:
import pandas
from pytz import timezone
from datetime import datetime, timedelta
from ipywidgets import interact, Dropdown
from plotly.offline import download_plotlyjs,init_notebook_mode, iplot
from plotly.graph_objs import Candlestick

from ReadH5 import ReadH5

init_notebook_mode(connected=True)

symbols = ['ES','TU','UB']
periods = {
'10 minutes': lambda dt: dt-timedelta(minutes=dt.minute%10, seconds=dt.second, microseconds=dt.microsecond),
'5 minutes': lambda dt: dt-timedelta(minutes=dt.minute%5, seconds=dt.second, microseconds=dt.microsecond),
'1 minute': lambda dt: dt-timedelta(seconds=dt.second, microseconds=dt.microsecond),
'30 seconds': lambda dt: dt-timedelta(seconds=dt.second%30, microseconds=dt.microsecond),
}
                                      
events = pandas.read_csv('../data/events.csv',names=['dt','curr','desc','imp','a','f','p','key'])
events['dt'] = pandas.to_datetime(events['dt'],format="%Y-%m-%dT%H:%M:%S").dt.tz_localize('UTC').dt.tz_convert("US/Central").dt.tz_localize(None)
grouped = events.groupby('key')

In [112]:
def plot(data, event, groupfunc):
    data['g'] = data['dt'].apply(groupfunc)
    ohlc = data.groupby(['g']).agg({'dt':'first','price':['first','max','min','last']})
    candles = Candlestick(x=ohlc.index
                          ,open=ohlc['price']['first']
                          ,high=ohlc['price']['max']
                          ,low=ohlc['price']['min']
                          ,close=ohlc['price']['last']
                         ,whiskerwidth=1)
    layout = {
    'title':event.desc,
    'shapes': [
    {
    'type': 'line', 'x0': event[0], 'y0': 0, 'x1': event[0], 'y1': 1,
    'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0, 0, 255)'}
    }
    ]
    }

    return iplot({'data':[candles],'layout':layout})
    
df = None
def load_data(symbol,dt,window=60): # window : one side window size in mins
    global df, reader
    if df is None or dt < reader.start_date or dt > reader.end_date:
        reader = ReadH5('../data/splits/',reportNoData=False)
        df = reader.readh5(symbol, dt)
    before = (dt - timedelta(minutes=window)).floor('D')
    after = (dt + timedelta(minutes=window)).ceil('D')
    return df[(before <= df["dt"]) & (df["dt"] <= after)].copy()

def onSelectEvent(*args):
    group = grouped.get_group(evdrop.value).set_index('dt')
    dtstrs = group.index.strftime('%b %d %Y, %r')
    dtdrop.value = None
    dtdrop.options = reversed(list(zip(dtstrs, group.itertuples())))
    
def run(sb,pd,_,ev):
    if ev is not None:
        data = load_data(sb,ev[0])
        return plot(data,ev,pd)

In [113]:
sbdrop = Dropdown(options=symbols,description="Security:",value='UB')
pddrop = Dropdown(options=periods,description="Periods:")
evdrop = Dropdown(options=grouped.groups.keys(),description="Event:",value=None);
dtdrop = Dropdown(description="Date:")
evdrop.observe(onSelectEvent, 'value')

_ = interact(run, sb=sbdrop, pd=pddrop, _=evdrop, ev=dtdrop)

interactive(children=(Dropdown(description='Security:', index=2, options=('ES', 'TU', 'UB'), value='UB'), Drop…